# refer https://zhuanlan.zhihu.com/p/168766486

In [1]:
import os

In [6]:
class Config(object):
    DEBUG = True
    SECRET_KEY = "*(%#4sxcz(^(#$#8423"
    # 数据库链接配置 = 数据库名称://登录账号:登录密码@数据库主机IP:数据库访问端口/数据库名称?charset=编码类型
    SQLALCHEMY_DATABASE_URI = os.environ.get('DEV_DATABASE_URL') or \
        'sqlite:///' + os.path.join('./', 'student.sqlite')

In [7]:
from flask import Flask
from config import Config

app = Flask(__name__,template_folder='templates')
app.config.from_object(Config)

In [17]:
app.config.from_object(Config)

# 初始化SQLAlchemy
from flask_sqlalchemy import SQLAlchemy
# db = SQLAlchemy() # 初始化数据库操作对象
# db.init_app(app)  # 初始化数据库链接d
db=SQLAlchemy(app)
class Student(db.Model):
    # 表结构声明
    __tablename__ = "tb_student"

    # 字段声明
    id   = db.Column(db.Integer, primary_key=True, comment="主键")
    name = db.Column(db.String(64), index=True, comment="姓名")
    sex  = db.Column(db.Boolean, default=True, comment="性别")
    age  = db.Column(db.SmallInteger, nullable=True, comment="年龄")
    email = db.Column(db.String(128), unique=True, comment="邮箱地址")
    money = db.Column(db.Numeric(8,2), default=0, comment="钱包")

    # 自定义方法
    def __repr__(self):
        return 'Student:%s' % self.name

class Teacher(db.Model):
    # 表结构声明
    __tablename__ = 'tb_teacher'

    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    option = db.Column(db.Enum("讲师","助教","班主任"), default="讲师")
    def __repr__(self):
        return 'Teacher:%s' % self.name

class Course(db.Model):
    # 定义表名
    __tablename__ = 'tb_course'
    # 定义字段对象
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    price = db.Column(db.Numeric(6,2))
    # repr()方法类似于django的__str__，用于打印模型对象时显示的字符串信息
    def __repr__(self):
        return 'Course:%s'% self.name

In [18]:
db.drop_all()

In [19]:
with app.app_context():
    db.create_all()
    # 创建数据
    # student = Student(name='张三',

In [20]:
student1=Student(name='张三',sex=True,age=17,email='12121@qq.com',money=100)
#with app.app_context():
db.session.add(student1)
db.session.commit()

In [21]:
student2 = Student(name='小红', sex=False, age=13, email="16565666@qq.com", money=600)
db.session.add(student2)
db.session.commit()

In [22]:
st1 = Student(name='wang',email='wang@163.com',age=22)
st2 = Student(name='zhang',email='zhang@189.com',age=22)
st3 = Student(name='chen',email='chen@126.com',age=22)
st4 = Student(name='zhou',email='zhou@163.com',age=22)
st5 = Student(name='tang',email='tang@163.com',age=22)
st6 = Student(name='wu',email='wu@gmail.com',age=22)
st7 = Student(name='qian',email='qian@gmail.com',age=22)
st8 = Student(name='liu',email='liu@163.com',age=22)
st9 = Student(name='li',email='li@163.com',age=22)
st10 = Student(name='sun',email='sun@163.com',age=22)
db.session.add_all([st1,st2,st3,st4,st5,st6,st7,st8,st9,st10])
db.session.commit()

In [ ]:
student=Student.query.first()
print(student)
db.session.delete(student)
db.session.commit()

In [24]:
ret=Student.query.filter_by(name='sun').delete()
print(ret)
db.session.commit()

1


In [25]:
# update datea
student=Student.query.first()
student.name='李四'
db.session.commit()

In [28]:
ret=Student.query.filter_by(name='李四').update({'money':1000})
db.session.commit()

ret=Student.query.filter(Student.age==22).update({Student.money: Student.money+100})
db.session.commit()

In [30]:
Student.query.get(2)

Student:李四

In [31]:
Student.query.all()

[Student:李四,
 Student:wang,
 Student:zhang,
 Student:chen,
 Student:zhou,
 Student:tang,
 Student:wu,
 Student:qian,
 Student:liu,
 Student:li]

In [32]:
ret=Student.query.filter(Student.name.endswith('g')).all()
ret=db.session.query(Student).filter(Student.age==22).all()

In [33]:
ret=Student.query.filter_by(name='李四').first()
ret=Student.query.filter_by(age=22).all()

In [ ]:
#查询所有男生数据
ret = Student.query.filter(Student.sex==True).all()
#查询所有女生数据
ret = Student.query.filter(Student.sex==False).all()
#查询id为4的学生[2种方式]
ret = Student.query.filter(Student.id==4).first()
ret = Student.query.get(4)
ret = Student.query.filter_by(id=4).first()    
#查询年龄等于22的所有学生数据
ret = Student.query.filter_by(age=22).all()
#查询name为liu的学生数据
ret = Student.query.filter(Student.name == "liu").all()
ret = Student.query.filter_by(name="liu").all()

In [34]:
Student.query.filter(Student.name.like("%i%")).all()
Student.query.filter(Student.name!="liu").all()

[Student:李四,
 Student:wang,
 Student:zhang,
 Student:chen,
 Student:zhou,
 Student:tang,
 Student:wu,
 Student:qian,
 Student:li]

In [36]:
from sqlalchemy import or_,not_,and_
Student.query.filter(or_(Student.name=="liu",Student.name=="wang")).all()
Student.query.filter(and_(Student.name=="liu",Student.email.endswith('163.com'))).all()
Student.query.filter(not_(Student.name=="liu")).all()

[Student:李四,
 Student:wang,
 Student:zhang,
 Student:chen,
 Student:zhou,
 Student:tang,
 Student:wu,
 Student:qian,
 Student:li]

In [37]:
student=Student.query.filter(Student.id.in_([1,2,3])).all()
print(student)

[Student:李四, Student:wang]


In [38]:
# order by

ret=Student.query.order_by(Student.age.desc()).all()
ret=Student.query.order_by(Student.age.asc(), Student.id.desc()).all()

In [39]:
#count
Student.query.filter(Student.age>20,Student.sex==True).count()
Student.query.filter(and_(Student.age>20,Student.sex==True)).count()

9

In [40]:
# 查询年龄最大的3个学生
ret1 = Student.query.order_by(Student.age.desc()).limit(3).all()

  # 查询年龄排第4到第7名的学生
ret2 = Student.query.order_by(Student.age.desc(),Student.id.desc()).offset(4).limit(4).all()
print(ret1,ret2)
# 查询名字和邮箱都以 li 开头的所有数据[2种方式]
ret = Student.query.filter(Student.name.startswith("li"),Student.email.startswith("li")).all()
# 查询age是 18 或者 `email` 以 `163.com` 结尾的所有学生
Student.query.filter(or_(Student.age==18,Student.email.endswith("163.com"))).all()
# 查询id为 [1, 3, 5, 7, 9] 的学生列表
student_list = Student.query.filter(Student.id.in_([1, 3, 5, 7, 9])).all()
print(student_list)
# 查询男生和女生的数量
ret = Student.query.filter_by(sex=True).count()
ret = Student.query.filter_by(sex=False).count()

[Student:wang, Student:zhang, Student:chen] [Student:tang, Student:zhou, Student:chen, Student:zhang]
[Student:wang, Student:chen, Student:tang, Student:qian]


In [43]:
# pagination
pagination = Student.query.paginate(per_page=3)
print(pagination.items)

# 获取当前页面所有数据
    # print( pagination.items )
    # data = {
    #     "items": [],
    #     "pages": pagination.pages,
    #     "page": pagination.page,
    #     "has_prev": pagination.has_prev,
    #     "has_next": pagination.has_next,
    # }

    # for item in pagination.items:
    #     data["items"].append({
    #         "id": item.id,
    #         "sex": "男" if item.sex else "女",
    #         "age": item.age,
    #         "name": item.name,
    #     })
    #
    # if pagination.has_prev:
    #     print( pagination.prev() ) # 上一页数据的分页器对象
    #     print( pagination.prev().items ) # 上一页数据
    #
    # if pagination.has_next:
    #     print( pagination.next() ) # 下一页数据的分页器对象
    #     print( pagination.next().items ) # 下一页数据

[Student:李四, Student:wang, Student:zhang]


In [44]:
# 查询当前所有男生女生的数量
# 第一个括号里面的是输出的内容， 第二个是分组条件
from sqlalchemy import func
    # ret = db.session.query(Student.sex,func.count(Student.id)).group_by(Student.sex).all()
    # 查询当前不同年龄的学生数量
ret = db.session.query(Student.age,func.count(Student.id)).group_by(Student.age).having(Student.age>19).all()

    # 查询男生和女生中，年龄最小的是几岁？
ret = db.session.query(Student.sex,func.min(Student.age)).group_by(Student.sex).all()

In [47]:
# 执行原生的sql 语句

ret=db.session.execute("select * from tb_student").fetchall()
ret=db.session.execute("select * from tb_student").fetchone()
db.session.execute("update tb_student SET money=(tb_student.money+%s) where tb_student.age=%s" %(200,22))
db.session.commit()